In [1]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
%aimport helpers
%autoreload 1

In [2]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from helpers import show_model
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

In [3]:
original_sentences = ['Mary Jane can see Will', \
                     'Spot will see Mary', \
                     'Will Jane spot Mary?', \
                     'Mary will pat Spot']

Next, let's clean the original sentences, remove punctuations, convert each word to Caps and remove duplicate words

In [4]:
import string

words = set()
for sentence in original_sentences:
    split = sentence.split()
    for word in split:
        x = word.translate(str.maketrans('', '',string.punctuation))
        x = x.capitalize()
        words.add(x)

In [5]:
words = list(words)
words

['Jane', 'Will', 'Can', 'Pat', 'Spot', 'Mary', 'See']

**Let's create the emmission_probs and change the order of words to conform to the video style:**<br>
Emission probability is the probability that a certain word is of a specific part of speech. E.G, Mary is a Noun.

In [6]:
emission_probs = {'noun':[4, 2, 1, 2, 0, 0, 0], \
                 'modal':[0, 0, 3, 0, 1, 0, 0], \
                 'verb':[0, 0, 0, 1, 0, 2, 1]}

words = ['Mary', 'Jane', 'Will', 'Spot', 'Can', 'See', 'Pat']
emissions_df = pd.DataFrame(emission_probs)
emissions_df.index = words

emissions_df

,noun,modal,verb
Mary,4,0,0
Jane,2,0,0
Will,1,3,0
Spot,2,0,1
Can,0,1,0
See,0,0,2
Pat,0,0,1


Now, to find the emission probabilities, let's divide each column by its sum

In [7]:
for i in emissions_df.columns:
    emissions_df[i] /= np.sum(emissions_df[i])

In [8]:
emissions_df

,noun,modal,verb
Mary,0.444444,0.00,0.00
Jane,0.222222,0.00,0.00
Will,0.111111,0.75,0.00
Spot,0.222222,0.00,0.25
Can,0.000000,0.25,0.00
See,0.000000,0.00,0.50
Pat,0.000000,0.00,0.25


**Let's also create the transition_probs and change the order of words to conform to the video style:**

Transition probability is the probability that a part of speech ike a Noun, follows another part, like a Verb.

In [9]:
transition_probs = {'noun':[3/4, 1/9, 1/4, 1], \
                 'modal':[1/4, 1/3, 0, 0], \
                 'verb':[0, 1/9, 3/4, 0], \
                   'tag':[0, 4/9, 0, 0]}

transitions_df = pd.DataFrame(transition_probs)
transitions_df.index = [transitions_df.columns[-1]] + list(transitions_df.columns[:-1])

transitions_df

,noun,modal,verb,tag
tag,0.750000,0.250000,0.000000,0.000000
noun,0.111111,0.333333,0.111111,0.444444
modal,0.250000,0.000000,0.750000,0.000000
verb,1.000000,0.000000,0.000000,0.000000


<h2>Hidden Markov Models:</h2>

Let's start assembling our HMM

In [10]:
# create the HMM model
model = HiddenMarkovModel(name="Speech")

In [11]:
# emission probability distributions, P(Word | Noun)
noun_dict = {i:round(j, 4) for i,j in zip(emissions_df.index, emissions_df.noun)}

noun_emissions = DiscreteDistribution(noun_dict)
noun_state = State(noun_emissions, name="Noun")

In [12]:
# emission probability distributions, P(Word | Modal)
modal_dict = {i:round(j, 4) for i,j in zip(emissions_df.index, emissions_df.modal)}

modal_emissions = DiscreteDistribution(modal_dict)
modal_state = State(modal_emissions, name="Modal")

In [13]:
# emission probability distributions, P(Word | Verb)
verb_dict = {i:round(j, 4) for i,j in zip(emissions_df.index, emissions_df.verb)}

verb_emissions = DiscreteDistribution(verb_dict)
verb_state = State(verb_emissions, name="Verb")

Now, lets add the states to the model...

In [14]:
# add the states to the model
model.add_states(noun_state, modal_state, verb_state)

### **IMPLEMENTATION:** Adding Transitions
Once the states are added to the model, we can build up the desired topology of individual state transitions.

#### Initial Probability $P(X_0)$:
We will assume that we don't know anything useful about the likelihood of a sequence starting in either state. If the sequences start on a Verb and end on a Noun (so each pattern is a new sequence). We can assign equal probability to each starting state by setting $P(X_0=Noun) = 0.3333$ and $P(X_0=Modal)=0.3333$ and $P(X_0=Verb)=0.3333$:

| $Noun$ | $Modal$ | $Verb$ |
| --- | --- | --- |
| 0.3333 | 0.3333 | 0.3333 |


In [15]:
transitions_df

,noun,modal,verb,tag
tag,0.750000,0.250000,0.000000,0.000000
noun,0.111111,0.333333,0.111111,0.444444
modal,0.250000,0.000000,0.750000,0.000000
verb,1.000000,0.000000,0.000000,0.000000


In [16]:
model.add_transition(model.start, noun_state, 0.750000)
model.add_transition(model.start, modal_state, 0.250000)
model.add_transition(model.start, verb_state, 0.000000)
model.add_transition(model.start, model.end, 0.000000)
model.add_transition(noun_state, noun_state, 0.111111)
model.add_transition(noun_state, modal_state, 0.333333)
model.add_transition(noun_state, verb_state, 0.111111)
model.add_transition(noun_state, model.end, 0.444444)
model.add_transition(modal_state, noun_state, 0.250000)
model.add_transition(modal_state, modal_state, 0.000000)
model.add_transition(modal_state, verb_state, 0.750000)
model.add_transition(modal_state, model.end, 0.000000)
model.add_transition(verb_state, noun_state, 1.000000)
model.add_transition(verb_state, modal_state, 0.000000)
model.add_transition(verb_state, verb_state, 0.000000)
model.add_transition(verb_state, model.end, 0.000000)
model.bake()

In [17]:
model.edge_count()

16

In [18]:
model.node_count()

5

**Visualize the Network**

In [19]:
show_model(model, figsize=(8, 8), filename="Speech.png", overwrite=True, show_ends=False)

FileNotFoundError: [WinError 2] "dot" not found in path.

<Figure size 576x576 with 0 Axes>

### Checking the Model
The states of the model can be accessed using array syntax on the `HMM.states` attribute, and the transition matrix can be accessed by calling `HMM.dense_transition_matrix()`. Element $(i, j)$ encodes the probability of transitioning from state $i$ to state $j$.

Run the next cell to inspect the full state transition matrix, then read the . 

In [ ]:
model.states

**Let's reorder the pomegranate default column orderings putting start first and last, last**

In [ ]:
column_order = ["Speech-start", "Modal", "Noun", "Verb", "Speech-end"]  # Override the Pomegranate default order
column_names = [s.name for s in model.states]
order_index = [column_names.index(c) for c in column_order]
print(order_index)

In [ ]:
# re-order the rows/columns to match the specified column order
transitions = model.dense_transition_matrix()[:, order_index][order_index, :]
print("The state transition matrix, P(Xt|Xt-1):\n")
print(transitions)
print("\nThe transition probability from Noun to Modal is {:.0f}%".format(100 * transitions[2, 1]))

In [ ]:
transitions[2, 1]

**The transition matrix here is arranged in the following fashion:**

1. The columns go down [Start, Modal, Noun, Verb, End]
2. The rows go across [Start, Modal, Noun, Verb, End] too.

<h2>Inference in Hidden Markov Models</h2>

#### IMPLEMENTATION: Calculate Sequence Likelihood

Calculating the likelihood of an observation sequence from an HMM network is performed with the [forward algorithm](https://en.wikipedia.org/wiki/Forward_algorithm). Pomegranate provides the the `HMM.forward()` method to calculate the full matrix showing the likelihood of aligning each observation to each state in the HMM, and the `HMM.log_probability()` method to calculate the cumulative likelihood over all possible hidden state paths that the specified model generated the observation sequence.

Fill in the code in the next section with a sample observation sequence and then use the `forward()` and `log_probability()` methods to evaluate the sequence.

In [ ]:
observations = ['Jane', 'Will', 'Spot', 'Will']
assert len(observations) > 0

In [ ]:
# TODO: use model.forward() to calculate the forward matrix of the observed sequence,
# and then use np.exp() to convert from log-likelihood to likelihood
forward_matrix = np.exp(model.forward(observations))
forward_matrix

In [ ]:
# TODO: use model.log_probability() to calculate the all-paths likelihood of the
# observed sequence and then use np.exp() to convert log-likelihood to likelihood
probability_percentage = np.exp(model.log_probability(observations))
probability_percentage

In [ ]:
# Display the forward probabilities
print("         " + "".join(s.name.center(len(s.name)+6) for s in model.states))
for i in range(len(observations) + 1):
    print(" <start> " if i==0 else observations[i - 1].center(9), end="")
    print("".join("{:.0f}%".format(100 * forward_matrix[i, j]).center(len(s.name) + 6)
                  for j, s in enumerate(model.states)))

print("\nThe likelihood over all possible paths " + \
      "of this model producing the sequence {} is {:.2f}%\n\n"
      .format(observations, 100 * probability_percentage))

### IMPLEMENTATION: Decoding the Most Likely Hidden State Sequence

The [Viterbi algorithm](https://en.wikipedia.org/wiki/Viterbi_algorithm) calculates the single path with the highest likelihood to produce a specific observation sequence. Pomegranate provides the `HMM.viterbi()` method to calculate both the hidden state sequence and the corresponding likelihood of the viterbi path.

This is called "decoding" because we use the observation sequence to decode the corresponding hidden state sequence. In the part of speech tagging problem, the hidden states map to parts of speech and the observations map to sentences. Given a sentence, Viterbi decoding finds the most likely sequence of part of speech tags corresponding to the sentence.

Fill in the code in the next section with the same sample observation sequence you used above, and then use the `model.viterbi()` method to calculate the likelihood and most likely state sequence. Compare the Viterbi likelihood against the forward algorithm likelihood for the observation sequence.

In [ ]:
# TODO: use model.viterbi to find the sequence likelihood & the most likely path
viterbi_likelihood, viterbi_path = model.viterbi(observations)

print("The most likely Speech sequence to have generated " + \
      "these observations is {} at {:.2f}%."
      .format([s[1].name for s in viterbi_path[1:]], np.exp(viterbi_likelihood)*100)
)

Let's see the most likely paths to generate all 4 original sentences using viterbi

In [ ]:
observations = ['Mary', 'Jane', 'Can', "See", 'Will']
viterbi_likelihood, viterbi_path = model.viterbi(observations)

print("The most likely Speech sequence to have generated " + \
      "these observations is {} at {:.2f}%."
      .format([s[1].name for s in viterbi_path[1:]], np.exp(viterbi_likelihood)*100)
)

In [ ]:
observations = ['Spot', 'Will', "See", 'Mary']
viterbi_likelihood, viterbi_path = model.viterbi(observations)

print("The most likely Speech sequence to have generated " + \
      "these observations is {} at {:.2f}%."
      .format([s[1].name for s in viterbi_path[1:]], np.exp(viterbi_likelihood)*100)
)

In [ ]:
observations = ['Will', "Jane", 'Spot', 'Mary']
viterbi_likelihood, viterbi_path = model.viterbi(observations)

print("The most likely Speech sequence to have generated " + \
      "these observations is {} at {:.2f}%."
      .format([s[1].name for s in viterbi_path[1:]], np.exp(viterbi_likelihood)*100)
)

In [ ]:
observations = ['Mary', 'Will', "Pat", 'Spot']
viterbi_likelihood, viterbi_path = model.viterbi(observations)

print("The most likely Speech sequence to have generated " + \
      "these observations is {} at {:.2f}%."
      .format([s[1].name for s in viterbi_path[1:]], np.exp(viterbi_likelihood)*100)
)